In [2]:
# Section 1: Imports
import time
from datetime import datetime
import re
from DrissionPage import errors
from DrissionPage import ChromiumPage
import mysql.connector
from mysql.connector import Error

def create_connection():
    try:
        connection = mysql.connector.connect(
            host='127.0.0.1',
            database='kol_1017',
            user='root',
            password='123456789',
            port=3306
        )
        if connection.is_connected():
            return connection
    except Error as e:
        print(f"Error while connecting to MySQL: {e}")
    return None

def fetch_instagram_links(connection):
    """Fetch all Instagram links from the kol_radar_info table."""
    try:
        cursor = connection.cursor()
        query = "SELECT kol_ig_link FROM kol_radar_info WHERE kol_ig_link IS NOT NULL"
        cursor.execute(query)
        links = cursor.fetchall()
        return [link[0] for link in links]  # Extract links from tuples
    except Error as e:
        print(f"Error: {e}")
        return []
    finally:
        if cursor:
            cursor.close()

In [3]:
def fetch_post_links(connection):
    """從 kol_post_data 表中提取所有 post_link 並分類 post 和 reel。"""
    try:
        cursor = connection.cursor()
        query = "SELECT post_link FROM kol_post_data WHERE post_link IS NOT NULL"
        cursor.execute(query)
        links = cursor.fetchall()
        return [link[0] for link in links]  # 提取連結文字
    except Error as e:
        print(f"Error: {e}")
        return []
    finally:
        if cursor:
            cursor.close()

# def fetch_account_id_by_post_link(connection, post_link):
#     """根據 post_link 從 kol_post_data 表中提取 account_id。"""
#     try:
#         cursor = connection.cursor()
#         query = "SELECT account_id FROM kol_post_data WHERE post_link = %s"
#         cursor.execute(query, (post_link,))
#         result = cursor.fetchone()  # 獲取單筆結果
#         return result[0] if result else None  # 如果有結果，返回 account_id，否則返回 None
#     except Error as e:
#         print(f"Error: {e}")
#         return None
#     finally:
#         if cursor:
#             cursor.close()

# import mysql.connector
# from mysql.connector import Error

def fetch_account_id_by_post_link(connection, post_link):
    """根據 post_link 從 kol_post_data 表中提取 account_id，並選擇 likes_count 最大的一筆記錄。"""
    try:
        with connection.cursor() as cursor:  # 確保 cursor 自動關閉
            query = "SELECT account_id, likes_count, comments_count FROM kol_post_data WHERE post_link = %s"
            cursor.execute(query, (post_link,))
            results = cursor.fetchall()  # 取得所有匹配的資料

            if results:
                # 根據 likes_count 選擇最大的一筆結果
                result = max(results)  
                return result[0] if result else None
            else:
                print(f"🔍 No results found for {post_link}")
                return None
    except Error as e:
        print(f"❌ Database Error: {e}")
        return None

    
    
def calculate_post_reel_ratio(links):
    """計算 post 和 reel 的數量及比例。"""
    post_count = sum(1 for link in links if '/p/' in link)
    reel_count = sum(1 for link in links if '/reel/' in link)
    total_count = len(links)

    if total_count == 0:
        return "無法計算比例，因為沒有資料。"

    post_ratio = (post_count / total_count) * 100
    reel_ratio = (reel_count / total_count) * 100

    result = {
        "總數": total_count,
        "Post 數量": post_count,
        "Reel 數量": reel_count,
        "Post 百分比": f"{post_ratio:.2f}%",
        "Reel 百分比": f"{reel_ratio:.2f}%"
    }

    return result

# 實際執行
connection = create_connection()
if connection:
    post_links = fetch_post_links(connection)
    result = calculate_post_reel_ratio(post_links)
    print(result)
    connection.close()

{'總數': 44937, 'Post 數量': 26297, 'Reel 數量': 18640, 'Post 百分比': '58.52%', 'Reel 百分比': '41.48%'}


In [4]:
def calculate_engagement_rate_by_post_link(connection, post_link):
    """
    根據貼文連結計算該貼文的「按讚數/粉絲數」指標。
    
    Args:
        connection: 資料庫連線物件。
        post_link: 貼文的連結。

    Returns:
        engagement_rate: 該貼文的按讚數/粉絲數比值。如果數據不全，返回 None。
    """
    cursor = None  # 初始化 cursor 變數
    try:
        cursor = connection.cursor()

        # 1. 查詢該貼文的按讚數和留言數及 account_id
        query_post = """
        SELECT likes_count, comments_count, account_id, post_key
        FROM kol_1017.kol_post_data
        WHERE post_link = %s
        """
        cursor.execute(query_post, (post_link,))
        post_data_list = cursor.fetchall()  # 抓取所有匹配的資料

        if not post_data_list:
            print("無法找到該貼文的數據")
            return None

        # 選擇按讚數最高的一筆資料
        post_data = max(post_data_list, key=lambda x: x[0])  # x[0] 是 likes_count

        likes_count, comments_count, account_id, post_key = post_data
        #print(f"likes_count: {likes_count}, comments_count: {comments_count}, account_id: {account_id}")
        # 2. 查詢該發文者的粉絲數
        query_account = """
        SELECT followers_num
        FROM kol_1017.accountinfo
        WHERE account_id = %s
        """
        cursor.execute(query_account, (post_key,))
        account_data = cursor.fetchone()

        if not account_data:
            print("無法找到該發文者的粉絲數")
            return None

        followers_num = account_data[0]

        if followers_num == 0:
            print("粉絲數為 0，無法計算 engagement rate")
            return None

        # 3. 計算 engagement rate (按讚數/粉絲數)
        engagement_rate = likes_count / followers_num

        print(f"按讚數: {likes_count}\n粉絲數: {followers_num}\n互動率 (按讚數/粉絲數): {engagement_rate:.4f}")

        return likes_count, comments_count, followers_num, engagement_rate

    except Error as e:
        print(f"Error: {e}")
        return None
    finally:
        if cursor is not None:
            cursor.close()  # 如果 cursor 被初始化，則關閉

In [5]:
# connection = create_connection()
# if connection:
#     # 取得所有符合條件的貼文連結
#     post_links = fetch_post_links(connection)
#     post_links = [link for link in post_links if '/p/' in link]
#     # 逐一計算每篇貼文的 engagement rate
#     for post_link in post_links[:1]:
#         calculate_engagement_rate_by_post_link(connection, post_link)


## 儲存圖片

In [6]:
import os
import requests

def save_image_from_url(image_url, file_name, kol_name, base_path='C:\\Users\\SHI\\Desktop\\KOL\\ig_photos'):
    """
    從圖片連結下載圖片並儲存在以 KOL 名稱為資料夾的目錄中。
    
    Args:
        image_url: 圖片的連結 URL。
        file_name: 儲存的檔案名稱（包含副檔名，如 image.jpg）。
        kol_name: KOL 名稱，用於建立子資料夾。
        base_path: 基本的儲存圖片路徑，預設為 'C:\\Users\\SHI\\Desktop\\KOL\\ig_photos'。
    """
    if not image_url:
        print(f"圖片連結無效，無法下載。")
        return
    
    try:
        # 組成 KOL 資料夾路徑
        save_path = os.path.join(base_path, kol_name)
        
        # 確保儲存目錄存在
        os.makedirs(save_path, exist_ok=True)

        # 儲存圖片檔案
        file_path = os.path.join(save_path, file_name)
        response = requests.get(image_url)
        response.raise_for_status()  # 確保請求成功
        with open(file_path, 'wb') as f:
            f.write(response.content)
        
        print(f"圖片已成功儲存至 {file_path}")
    except Exception as e:
        print(f"下載圖片時出現錯誤: {e}")

In [7]:
def insert_post_data(connection, post_data):
    """
    將貼文資料插入到資料庫的 kol_1017.kol_post_detail 表中。
    
    Args:
        connection: 資料庫連線物件。
        post_data: 字典形式的貼文資料。
    """
    try:
        cursor = connection.cursor()

        # 插入資料的 SQL 語句
        insert_query = """
        INSERT INTO kol_1017.kol_post_detail (post_link, post_id, author, cooperator, article, hashtags, 
                                              product_links, mentions, post_time, image_url, likes_count, followers_count, engagement_rate)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(insert_query, (
            post_data['post_link'], post_data['post_id'], post_data['author'], post_data['cooperator'], post_data['article'],
            post_data['hashtags'], post_data['product_links'], post_data['mentions'], post_data['post_time'], post_data['image_url'],
            post_data['likes_count'], post_data['followers_count'], post_data['engagement_rate']
        ))

        # 提交變更
        connection.commit()
        print(f"貼文資料已成功插入資料庫 (Post ID: {post_data['post_id']})")
    except Exception as e:
        print(f"插入資料時發生錯誤: {e}")
    finally:
        if cursor:
            cursor.close()

def insert_comments(connection, post_id, comments):
    """
    將留言內容插入資料庫的 kol_post_comments 表。

    Args:
        connection: 資料庫連線物件。
        post_id: 貼文的 post_id。
        comments: 留言列表，格式為 [(留言流水號, 留言者 IG ID, 留言內容), ...]。
    """
    try:
        cursor = connection.cursor()

        # 插入留言的 SQL 語句
        insert_query = """
        INSERT INTO kol_1017.kol_post_comments (post_id, comment_id, commenter_id, comment_text)
        VALUES (%s, %s, %s, %s)
        """

        # 插入每一則留言
        for comment_id, commenter_id, comment_text in comments:
            cursor.execute(insert_query, (post_id, comment_id, commenter_id, comment_text))

        # 提交變更
        connection.commit()
        print(f"成功插入 {len(comments)} 則留言到資料庫")
    except Exception as e:
        print(f"插入留言時發生錯誤：{e}")
    finally:
        if cursor:
            cursor.close()



##  抓取資料


In [8]:
from datetime import datetime
import re
import time

def scrape_posts(connection, post_links, start_index, end_index):
    failed_posts = []  # 紀錄未成功的貼文連結和原因
    processed_count = 0  # 計數處理的貼文數量
    rest_interval = 100 # 每處理多少篇貼文後休息
    rest_time =  100

    page = ChromiumPage()

    for link in post_links[start_index:end_index]:
        try:
            page.get(link)
            page.wait(1)

            # 檢查是否為影片
            video_elements = page.eles('.x1lliihq x5yr21d xh8yej3')
            if video_elements:
                print(f"第 {post_links.index(link)+1} 篇貼文是影片，跳過")
                continue
            else:
                print(f"第 {post_links.index(link)+1} 篇貼文")

                post_id = fetch_account_id_by_post_link(connection, link)
                print(f"Post Link: {link}\nPost 編號: {post_id}")

                post_people = page.eles('.x9f619 xjbqb8w x78zum5 x168nmei x13lgxp2 x5pf9jr xo71vjh x1uhb9sk x1plvlek xryxfnj x1c4vz4f x2lah0s x1q0g3np xqjyukv x1cy8zhl x1oa3qoh x1nhvcw1')

                if post_people:
                    kol = post_people[2].text if len(post_people) > 2 else ""  
                    cooperator = post_people[4].text if len(post_people) > 4 else ""  
                else:
                    backup_elements = page.eles('.x9f619 xjbqb8w x78zum5 x168nmei x13lgxp2 x5pf9jr xo71vjh x1n2onr6 x1plvlek xryxfnj x1c4vz4f x2lah0s x1q0g3np xqjyukv x6s0dn4 x1oa3qoh x1nhvcw1')
                    kol = backup_elements[0].text if backup_elements else ""  
                    cooperator = ""

                if '\n' in kol:
                    kol = re.sub(r'\n.*$', '', kol).strip()

                post_article_element = page.ele('._ap3a _aaco _aacu _aacx _aad7 _aade')
                post_article = post_article_element.text if post_article_element else ""

                hashtags = ', '.join(sorted(set(re.findall(r'#\w+', post_article))))
                product_links = ', '.join(sorted(set(re.findall(r'https?://\S+', post_article))))
                mentions = ', '.join(sorted(set(re.findall(r'@[a-zA-Z0-9._]+', post_article))))

                post_time = page.ele('.x1p4m5qa', timeout=5).attr('datetime')
                if post_time:
                    try:
                        dt = datetime.strptime(post_time, "%Y-%m-%dT%H:%M:%S.%fZ")
                        post_datetime = dt.strftime("%Y-%m-%d %H:%M:%S")
                    except ValueError:
                        print(f"無法解析日期時間: {post_time}")
                        post_datetime = None

                image = page.ele('.x5yr21d xu96u03 x10l6tqk x13vifvy x87ps6o xh8yej3').attr('src')
                if image:
                    file_name = f"{post_id}.jpg"
                    save_image_from_url(image, file_name, kol)
                likes_count, comments_count, followers_num, engagement_rate = calculate_engagement_rate_by_post_link(connection, link)  
                post_data = {
                    "post_link": link,
                    "post_id": post_id,
                    "author": kol,
                    "cooperator": cooperator,
                    "article": post_article,
                    "hashtags": hashtags,
                    "product_links": product_links,
                    "mentions": mentions,
                    "post_time": post_datetime,
                    "image_url": image,
                    "likes_count": likes_count,
                    "followers_count": followers_num,
                    "engagement_rate": engagement_rate
                }

                insert_post_data(connection, post_data)

                processed_count += 1

                # 每處理到 rest_interval 篇時，休息一段時間
                if processed_count % rest_interval == 0:
                    print(f"🌙 已處理 {processed_count} 篇貼文，休息 {rest_time / 60:.1f} 分鐘...")
                    time.sleep(rest_time)

        except Exception as e:
            post_id = "無法取得貼文編號"
            failed_posts.append((link, str(e)))
            print(f"跳過貼文 {link}，錯誤原因：{e}")

    print(f"共處理 {processed_count} 篇貼文")
    return failed_posts

11228開始暫停抓留言

In [9]:
#呼叫函數
connection = create_connection()
if connection:
    post_links = fetch_post_links(connection)
    post_links = [link for link in post_links if '/p/' in link]

scrape_posts(connection, post_links, 25563, 26500)

第 25564 篇貼文
Post Link: https://www.instagram.com/zrbros/p/C4pmjycvoSc/
Post 編號: zrbros_019
圖片已成功儲存至 C:\Users\SHI\Desktop\KOL\ig_photos\zrbros\zrbros_019.jpg
按讚數: 1999
粉絲數: 45000
互動率 (按讚數/粉絲數): 0.0444
貼文資料已成功插入資料庫 (Post ID: zrbros_019)
第 25565 篇貼文是影片，跳過
第 25566 篇貼文
Post Link: https://www.instagram.com/zrbros/p/C3eJIQevTX8/
Post 編號: zrbros_021
圖片已成功儲存至 C:\Users\SHI\Desktop\KOL\ig_photos\zrbros\zrbros_021.jpg
按讚數: 270
粉絲數: 45000
互動率 (按讚數/粉絲數): 0.0060
貼文資料已成功插入資料庫 (Post ID: zrbros_021)
第 25567 篇貼文
Post Link: https://www.instagram.com/zrbros/p/C3dBBn7PVaH/
Post 編號: zrbros_022
圖片已成功儲存至 C:\Users\SHI\Desktop\KOL\ig_photos\zrbros\zrbros_022.jpg
按讚數: 953
粉絲數: 45000
互動率 (按讚數/粉絲數): 0.0212
貼文資料已成功插入資料庫 (Post ID: zrbros_022)
第 25568 篇貼文
Post Link: https://www.instagram.com/zrbros/p/C3Y07yCvk5l/
Post 編號: zrbros_024
圖片已成功儲存至 C:\Users\SHI\Desktop\KOL\ig_photos\zrbros\zrbros_024.jpg
按讚數: 312
粉絲數: 45000
互動率 (按讚數/粉絲數): 0.0069
貼文資料已成功插入資料庫 (Post ID: zrbros_024)
第 25569 篇貼文
Post Link: https://www.instagram.

[]

IG 怎麼推播他的貼文

預測的模型要多嘗試幾個